In [1]:
import os
import sys

import warnings
warnings.filterwarnings("ignore") 

os.environ['UTILS_PATH'] = r'C:\Users\Mi\Desktop\code\git\python-utils-ak'
os.environ['PROJECT_PATH'] = r'C:\Users\Mi\Desktop\code\git\2020.10-umalat\umalat' 
os.environ['SQLITE_PATH'] = r'C:\Users\Mi\Desktop\code\git\2020.10-umalat\umalat\data.sqlite' 
sys.path.append(os.environ['PROJECT_PATH'])
from app.interactive_imports import *

In [2]:
# # boiling_plan_df = read_boiling_plan(r"C:\Users\Mi\Desktop\code\git\2020.10-umalat\umalat\research\akadaner\2021-01-06 (5).xlsx")
# boiling_plan_df = read_boiling_plan(r"C:\Users\Mi\Desktop\code\git\2020.10-umalat\umalat\app\schedule_maker\data\sample_boiling_plan.xlsx")
# boilings = make_boilings_by_groups(boiling_plan_df)
# schedule = make_schedule(boilings, start_times={'water': '08:00', 'salt': '07:00'})
# frontend = make_frontend(schedule)
# # draw_excel_frontend(frontend, open_file=True)

In [3]:
from app.schedule_maker import read_boiling_plan
# df = read_boiling_plan(r"../data/sample_boiling_plan.xlsx")
df = read_boiling_plan(r"C:\Users\Mi\Desktop\code\git\2020.10-umalat\umalat\app\schedule_maker\data\sample_boiling_plan.xlsx")

mark_consecutive_groups(df, 'boiling', 'boiling_group')
boiling_group_df = df[df['boiling_group'] == 2]

boilings_meltings, packings, melting_speed = BoilingGroupToSchemaTransformer()(boiling_group_df)
boilings_dataframes = SchemaToBoilingsDataFramesTransformer()(boilings_meltings, packings, melting_speed)
boilings_dataframes

[{'meltings':           item   beg   end
  0  cheese_mass   0.0   5.0
  1  cheese_mass  10.0  70.0,
  'coolings':                      item   beg    end
  0   <BoilingFormFactor 3>   0.0   90.0
  1  <BoilingFormFactor 13>  95.0  145.0,
  'packings': [       item    beg    end
   0  <SKU 12>   90.0   95.0
   1  <SKU 25>  100.0  160.0]},
 {'meltings':           item  beg   end
  0  cheese_mass  0.0  65.0,
  'coolings':                     item  beg    end
  0  <BoilingFormFactor 4>  0.0  140.0,
  'packings': [       item   beg    end
   0  <SKU 30>  75.0  145.0]},
 {'meltings':           item  beg   end
  0  cheese_mass  0.0  65.0,
  'coolings':                     item  beg    end
  0  <BoilingFormFactor 4>  0.0  140.0,
  'packings': [       item   beg    end
   0  <SKU 30>  75.0  145.0]}]

In [4]:
boiling_dataframes = boilings_dataframes[0]
boiling_dataframes

{'meltings':           item   beg   end
 0  cheese_mass   0.0   5.0
 1  cheese_mass  10.0  70.0,
 'coolings':                      item   beg    end
 0   <BoilingFormFactor 3>   0.0   90.0
 1  <BoilingFormFactor 13>  95.0  145.0,
 'packings': [       item    beg    end
  0  <SKU 12>   90.0   95.0
  1  <SKU 25>  100.0  160.0]}

In [5]:
ERROR = 1e-5

In [42]:
def make_line(df, line_name, process_name, filler_name, item_name):
    maker, make = init_block_maker(line_name)
    
    for i in range(len(df)):
        cur_row = df.iloc[i]
        next_row = None if i == len(df) - 1 else df.iloc[i + 1] # todo: use simple_bounded_iterator? 

        # todo: refactor item_name, make general key item in process? 
        make(process_name, size=((cur_row['end'] - cur_row['beg']) // 5, 0), x=(cur_row['beg'] // 5, 0), push_func=add_push, **{item_name: cur_row['item']})

        # fillers
        if next_row is not None:
            beg = cur_row['end']
            end = next_row['beg']
            if abs(beg - end) > ERROR:
                make(filler_name, size=((end - beg) // 5, 0))
    return maker.root

In [43]:
make_line(boiling_dataframes['meltings'], 'meltings', 'melting_process', 'melting_configuration', 'bff')

============== meltings (0, 14.0] x (0, 0]
= melting_process (0.0, 1.0] x (0, 0]
 = melting_configuration (1.0, 2.0] x (0, 0]
  ============ melting_process (2.0, 14.0] x (0, 0]

In [44]:
boiling_model = boiling_group_df.iloc[0]['boiling']
boiling_type = boiling_group_df.iloc[0]['boiling'].boiling_type
melting_speed = 900 if boiling_type == 'water' else 1020  # todo: duplicate with s1

maker, make = init_block_maker('melting_and_packing')

with make('melting'):
    serving = make('serving', size=(boiling_model.meltings.serving_time // 5, 0), push_func=add_push).block
    
    line = make_line(boiling_dataframes['meltings'], 'meltings', 'melting_process', 'melting_configuration', 'bff')
    line.props.update({'x': (serving.size[0], 0)})
    push(maker.root['melting'], line, push_func=add_push)
    
    line = make_line(boiling_dataframes['coolings'], 'coolings', 'cooling_process', 'cooling_configuration', 'bff')
    line.props.update({'x': (serving.size[0], 0)})
    push(maker.root['melting'], line, push_func=add_push)


for packing_team_id, df in enumerate(boiling_dataframes['packings']):
    line = make_line(df, 'packing', 'packing_process', 'packing_configuration', 'sku')
    line.props.update({'x': (serving.size[0], 0)})
    push(maker.root, line, push_func=add_push)

In [45]:
maker.root

====================================== melting_and_packing (0, 38.0] x (0, 0]
=================================== melting (0, 35.0] x (0, 0]
====== serving (0, 6] x (0, 0]
      ============== meltings (6, 20.0] x (0, 0]
      = melting_process (6.0, 7.0] x (0, 0]
       = melting_configuration (7.0, 8.0] x (0, 0]
        ============ melting_process (8.0, 20.0] x (0, 0]
      ============================= coolings (6, 35.0] x (0, 0]
      ================== cooling_process (6.0, 24.0] x (0, 0]
                        = cooling_configuration (24.0, 25.0] x (0, 0]
                         ========== cooling_process (25.0, 35.0] x (0, 0]
      ================================ packing (6, 38.0] x (0, 0]
                        = packing_process (24.0, 25.0] x (0, 0]
                         = packing_configuration (25.0, 26.0] x (0, 0]
                          ============ packing_process (26.0, 38.0] x (0, 0]